In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint

from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from auxiliar_func import *
from plot_func import *

In [2]:
df = pd.read_csv('Census-Income-KDD.csv')
target = 'income_50k'
df_tr, df_te = train_test_split(df, test_size=0.3, random_state=42)

## Metrics of the final catboost model

In [5]:
prep_params = {
    'scaling': None,
    'imputation': 'mode',
    'cat_age': False,
    'target_freq': 0.8,
    'generate_dummies': False
}

# a first preprocess to get the categorical features
df_tr_pre = preprocessing(df_tr, scaling=None, imputation='mode', cat_age=False, target_freq=0.8, generate_dummies=False)
X_train, y_train = df_tr_pre.drop(target, axis=1), df_tr_pre[target]
cat_features = list(X_train.select_dtypes(include=['category']).columns)

cat_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
    logging_level='Silent',
    cat_features=cat_features,
    verbose=False
)

pprint(cross_validation(cat_model, df_tr, prep_params, cv=5))

Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU


{'accuracy': 0.947338650770351,
 'f1_macro': 0.7941872977686797,
 'precision_macro': 0.7737583673527599,
 'recall_macro': 0.8190847030701803}
